In [ ]:
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.agents.file_surfer import FileSurfer
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from dotenv import load_dotenv
from pathlib import Path

In [ ]:
# Get the absolute path to the .env file
env_path = Path(os.getcwd()).parent / ".env"

# Load the .env file
load_dotenv(dotenv_path=env_path)

In [ ]:
# Get environment variable
AZURE_AI_SERVICE_NAME = os.getenv("AZURE_AI_SERVICE_NAME")

# Create the token provider
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o",
    api_version="2024-05-01-preview",
    model="gpt-4o",
    azure_endpoint=f"https://{AZURE_AI_SERVICE_NAME}.openai.azure.com/",
    azure_ad_token_provider=token_provider,
)

In [7]:
def read_file_tool(filePath: str):
    try:
        return file_tool("read", None, filePath)
    except Exception as e:
        return str(e)

def file_tool(operation: str, body: str, filePath: str):
    """
    Useful tool to read or write files
    
    :param operation: Operation to execute
    :param body: Body of the file
    :param filePath: Path to the file
    :return: message
    """
    try:
        if operation == "read":
            with open(filePath, "r") as f:
                return f.read()
        elif operation == "write":
            with open(filePath, "w") as f:
                f.write(body)
                return f"File {filePath} written successfully"
        else:
            raise ValueError("Invalid operation")
    except Exception as e:
        return str(e)

def directory_write_tool(directory_path: str):
    """
    Useful to create a directory with the given path.
    
    :param directory_path: Path to the directory
    :return: message
    """
    try:
        if not os.path.exists(directory_path):
            os.makedirs(directory_path)
            return f"Directory '{directory_path}' has been created successfully."
        else:
            return f"Directory '{directory_path}' already exists."
    except Exception as e:
        return str(e)

def list_all_files(directory: str):
    """
    Recursively lists all files in the given directory and its subdirectories.
    
    :param directory: Path to the directory
    :return: List of file paths
    """
    file_list = []

    try:
        for root, _, files in os.walk(directory):
            for file in files:
                file_list.append(os.path.join(root, file))
    except Exception as e:
        return str(e)
    
    return file_list

def delete_file(file_path: str):
    """
    Deletes the specified file.
    
    :param file_path: Path to the file to be deleted
    """
    try:
        os.remove(file_path)
        print(f"Deleted: {file_path}")
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except PermissionError:
        print(f"Permission denied: {file_path}")
    except Exception as e:
        print(f"Error deleting file {file_path}: {e}")

In [10]:
tools = [file_tool, directory_write_tool, list_all_files]
super_tools = [file_tool, directory_write_tool, list_all_files, delete_file]

# Create the developer agent.
developer_agent = AssistantAgent(
    name="developer",
    model_client=az_model_client,
    system_message=
    """
    Your final answer must include the python code implementing the solution.
    You have access to tools which can read files, write files and create directories.
    """,
    tools=[tool for tool in tools]
)

# Create the developer agent.
tester_agent = AssistantAgent(
    name="tester",
    model_client=az_model_client,
    system_message=
    """
    You are a test engineer.
    Write and run test cases for the code implemented by the developer.
    You have access to tools which can read files, write files and create directories.
    """,
    tools=[tool for tool in tools]
)

# Create the tech-lead agent.
techlead_agent = AssistantAgent(
    name="techlead",
    model_client=az_model_client,
    system_message=
    """
    You are a technical lead.
    You must challenge the code and provide constructive feedback according to the best practices.
    Don't allow multiple classes in a single file. Split them into separate files.
    Create markdown documentation is always produced.
    Delete files in the directory created that are not of any use.
    If you are happy with the outcome, respond with 'FINISH' to terminate the conversation.
    """,
    tools=[tool for tool in super_tools]
)

In [11]:
# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("FINISH")

# Create a team with the primary and critic agents.
team = SelectorGroupChat([developer_agent, tester_agent, techlead_agent], model_client=az_model_client, termination_condition=text_termination)

In [ ]:
task="""
There is a folder DotNet8WebApiProject that contains a dotnet application.
Create a FastAPI template project and migrate the dotnet application to python.
"""
await Console(team.run_stream(task=task))